In [ ]:
!pip install boto3 langchain -Uq
!pip install --upgrade pip
!pip install langchain_community
!pip install -U langchain-aws


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 705.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.9/379.9 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.2/140.2 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 7.3 MB/s eta 0:00:00
  Using cached pip-24.2-py3-none-any.whl.metadata (3.6 kB)
Using cached pip-24.2-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 16.4 MB/s eta 0:00:00


In [ ]:
import langchain
import langchain_community
#import langchain_aws
#print(langchain.__version__)
#print(langchain_community.__version__)

In [ ]:
import boto3
import json
import socket
import pandas as pd

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import BedrockChat
from langchain.prompts import PromptTemplate # Import PromptTemplate
from langchain.llms import Bedrock # Import the Bedrock class from langchain.llms


In [ ]:
def setSettings(access_key,secret_key,region):
  AWS_ACCESS_KEY = access_key
  AWS_SECRET_KEY = secret_key
  REGION = region

  session = boto3.Session(
                          aws_access_key_id=AWS_ACCESS_KEY,
                          aws_secret_access_key=AWS_SECRET_KEY,
                          region_name=REGION
                          )

  bedrock_runtime = session.client('bedrock-runtime')

  return bedrock_runtime


In [ ]:
bedrock_runtime= setSettings("your access key", "your secret access key",'your region')

In [ ]:
def getPredictedCategory(model,bedrock,final_prompt):
    # Initilization
    instruction="You are a scientific researcher that needs to perform the following task."
    llama3='''<|begin_of_text|><|start_header_id|>user<|end_header_id|>
              [message]<|eot_id|><|start_header_id|>assistant<|end_header_id|>'''
    # Model setting
    if model == "llama2-70b":
        model_id = "meta.llama2-70b-chat-v1"
        model_kwargs = {"max_gen_len":2048, "temperature":0.1, "top_p":1}
        instruction="<s>[INST] <<SYS>>\n"+instruction+"\n<</SYS>>\n"
        final_prompt = final_prompt+" [/INST]"
    elif model == "llama3-70b":
       model_id = "meta.llama3-70b-instruct-v1:0"
       model_kwargs = {"max_gen_len":500, "temperature":0.1, "top_p":1}
       #instruction="<s>[INST] <<SYS>>\n"+instruction+"\n<</SYS>>\n"
       final_prompt = llama3.replace("[message]", final_prompt)
    elif model == "llama3-8b":
       model_id = "meta.llama3-8b-instruct-v1:0"
       model_kwargs = {"max_gen_len":500, "temperature":0.1, "top_p":1}
       #instruction="<s>[INST] <<SYS>>\n"+instruction+"\n<</SYS>>\n"
       final_prompt = llama3.replace("[message]", final_prompt)
    elif model == "mistral-7b-instruct":
       model_id = "mistral.mistral-7b-instruct-v0:2"
       model_kwargs = {"max_tokens":2048, "temperature":0.1, "top_p":1, "top_k":100}
       #final_prompt = "<s>[INST] "+final_prompt+" [/INST]"
    elif model == "mistral-8x7b-instruct":
       model_id = "mistral.mixtral-8x7b-instruct-v0:1"
       model_kwargs = {"max_tokens":2048, "temperature":0.1, "top_p":1, "top_k":100}
       final_prompt = "<s>[INST] "+final_prompt+" [/INST]"
    elif model == "mistral-large":
       model_id = "mistral.mistral-large-2402-v1:0"
       model_kwargs = {"max_tokens":2048, "temperature":0.1, "top_p":1, "top_k":100}
       final_prompt = "<s>[INST] "+final_prompt+" [/INST]"
    elif model == "cohere":
       model_id = "cohere.command-text-v14"
       model_kwargs = {"max_tokens":2048, "temperature":0.1, "p":1, "k":100}
    elif model == "cohere-light":
       model_id = "cohere.command-light-text-v14"
       model_kwargs = {"max_tokens":2048, "temperature":0.1, "p":1, "k":100}
    elif model == "claude-haiku":
         model_id = "anthropic.claude-3-haiku-20240307-v1:0"
         model_kwargs =  {"max_tokens": 2048, "temperature": 0.1,"top_k": 100, "top_p": 1,   "stop_sequences": ["\n\nHuman"], }
    elif model == "claude-sonnet":
         model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
         model_kwargs =  {"max_tokens": 2048, "temperature": 0.1,"top_k": 100, "top_p": 1,   "stop_sequences": ["\n\nHuman"], }
    else:
            raise Exception("Error with the MODEL parameter. Valid values are llama, mistral , sonnet or cohere")

     # Model Invoke
    model = BedrockChat(
                        client=bedrock,
                        model_id=model_id,
                        model_kwargs=model_kwargs
                        )
    # Invoke Example
    messages = [
                  ("system", instruction),
                  ("human", "{question}")
                ]

    prompt = ChatPromptTemplate.from_messages(messages)
    chain = prompt | model | StrOutputParser()

      # Chain Invoke
    response = chain.invoke({"question": final_prompt})

    return response


In [ ]:
df = pd.read_excel("Sentence_Classification_Raters_SyntheticData_Cleaned_Levenshtein_v01.xlsx")

df

,Id,Sentence,Synthetic_Sentence_1,Synthetic_Sentence_2,Synthetic_Sentence_3,Synthetic_Sentence_4,Synthetic1_Orginal_Normalized_Levenshtein,Synthetic1_Other_Normalized_Levenshtein,Synthetic2_Orginal_Normalized_Levenshtein,Synthetic2_Other_Normalized_Levenshtein,Synthetic3_Orginal_Normalized_Levenshtein,Synthetic3_Other_Normalized_Levenshtein,Synthetic4_Original_Normalized_Levenshtein,Synthetic4_Other_Normalized_Levenshtein
0,RecSys 02 DES 4,in our recent work on music genre exploration ...,In our latest research on musical genre explor...,Our recent study on music genre exploration [1...,In our latest work related to exploring music ...,Our recent research project on music genre exp...,0.436464,0.478394,0.401042,0.505015,0.442211,0.573687,0.468750,0.449405
1,RecSys 04 DES 3,in order to reflect the users complete spectru...,To capture the diverse reading preferences of ...,Recognizing that readers have varied interests...,With the goal of mirroring the full breadth of...,To reflect the diverse topical interests of re...,0.446701,0.448821,0.511628,0.624963,0.434579,0.517735,0.458716,0.480122
2,RecSys 05 DES 1,mcauley et al. [21] mine key attributes from t...,[21] employs topic modeling techniques to extr...,The study by mcauley et al. [21] utilizes topi...,mcauley et al. [21] leverage topic extraction ...,The researchers in [21] apply topic extraction...,0.744444,0.522813,0.613445,0.445378,0.626168,0.442631,0.669643,0.470588
3,CHI 01 DES 3,"beyond inspection, chickenfoot [7] allows end-...","Transcending mere examination, Chickenfoot [7]...","Surpassing the confines of mere inspection, Ch...","Extending beyond the realm of mere inspection,...",Transcending the boundaries of mere inspection...,0.359375,0.372514,0.404682,0.415831,0.367003,0.386237,0.346715,0.339009
4,CHI 01 DES 5,aimed at designers who have limited coding kno...,Targeting designers with minimal coding expert...,Designed for designers with restricted coding ...,Catering to designers with limited coding skil...,Aimed at designers with constrained coding abi...,0.354680,0.309728,0.266355,0.321624,0.319635,0.315068,0.244240,0.273994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,sen_33,"The question-order effect (i.e., one question ...",The sequence of questions can influence respon...,One query may establish a context that guides ...,The order in which questions are presented can...,The question-order effect refers to the potent...,0.709220,0.588802,0.698113,0.648374,0.712707,0.589319,0.567073,0.717817
136,sen_15,The main limitation is that the experiments ar...,The primary constraint lies in the fact that t...,The study's scope is restricted to the stereot...,The experiments' foundation rests exclusively ...,The research's confines stem from its reliance...,0.470199,0.688742,0.664234,0.537516,0.605634,0.572024,0.604167,0.528406
137,sen_55,"However, special care must be taken to avoid b...","Nonetheless, precautions are necessary to prev...",Particular attention should be paid to mitigat...,Careful measures must be implemented to circum...,Specific steps are required to eliminate the p...,0.633663,0.615464,0.655914,0.596349,0.647619,0.565079,0.666667,0.573749
138,sen_10,"Hence, it is possible that a more detailed cal...","Consequently, a more comprehensive calibration...",It is conceivable that a more thorough calibra...,A more intricate calibration procedure may pot...,"Therefore, the possibility exists that a more ...",0.681818,0.683050,0.479167,0.642491,0.704545,0.729876,0.411215,0.663551


In [ ]:
prompt= '''\Text Paraphrasing Instructions:\
PROCEDURE:\
1. Rephrase the given text preserving the main ideas and context.\
2. Gives 4 different answers. Each answer should start and end with *.\

EXAMPLE:\
Text: Today is a sunny day.\
Output for the text: *The weather is bright and cloudless on this day.* *Sunlight bathes the landscape as we enjoy clear skies.* *Warm rays illuminate our surroundings on this radiant afternoon.* *The sun's cheerful presence defines the atmosphere today.*\

Text to Paraphrase:[sentence].'''

In [ ]:
#getSyntheticText("claude-sonnet",bedrock_runtime,prompt)

In [ ]:
# Iterate the dataframe
def setSyntheticText(df,model,bedrock,prompt):
  for index, row in df.iterrows():
      prompt2 = row["Sentence"]
      final_prompt = prompt.replace("[sentence]", prompt2)
      df.loc[index, "Synthetic_Sentence"] =getSyntheticText(model,bedrock,final_prompt)
      #print(f"{r}\n\n{final_prompt}")
      print(f"{row['Id']}:  {getSyntheticText(model,bedrock,final_prompt)}")


In [ ]:
setSyntheticText(df, "claude-sonnet",bedrock_runtime,prompt)
#setSyntheticText(top_3_rows, "claude-sonnet",bedrock_runtime,prompt)

RecSys 02 DES 4:  *In our latest research on musical genre exploration [13], we implemented various methods to assist users in navigating different genres, primarily utilizing decision structure nudges.*

*Our recent study on music genre exploration [13] involved the application of several techniques, with a focus on decision structure nudges, to facilitate users' exploration of various genres.*

*In our recent work [13] related to exploring music genres, we employed multiple approaches, predominantly decision structure nudges, to support users in their journey of genre discovery.*

*Our latest endeavor [13] in the realm of music genre exploration involved the utilization of several techniques, with a particular emphasis on decision structure nudges, to aid users in their exploration of different genres.*
RecSys 04 DES 3:  *To capture the diverse reading preferences of users, Ziegler et al. [38] introduced the concept of topic diversification, which aims to balance and diversify person

In [ ]:
df
#top_3_rows

,Id,Sentence,Fco,Pablo,Paula,Vote,Unique,Unique_Vote,Synthetic_Sentence
0,RecSys 02 DES 4,in our recent work on music genre exploration ...,DESCRIPTION,DESCRIPTION,DESCRIPTION,DESCRIPTION,YES,YES,*In our latest research on musical genre explo...
1,RecSys 04 DES 3,in order to reflect the users complete spectru...,DESCRIPTION,DESCRIPTION,DESCRIPTION,DESCRIPTION,YES,YES,*To capture the diverse reading preferences of...
2,RecSys 05 DES 1,mcauley et al. [21] mine key attributes from t...,DESCRIPTION,DESCRIPTION,DESCRIPTION,DESCRIPTION,YES,YES,*[21] employs topic modeling techniques to ext...
3,CHI 01 DES 3,"beyond inspection, chickenfoot [7] allows end-...",DESCRIPTION,DESCRIPTION,DESCRIPTION,DESCRIPTION,YES,YES,"*Transcending mere examination, Chickenfoot [7..."
4,CHI 01 DES 5,aimed at designers who have limited coding kno...,DESCRIPTION,DESCRIPTION,DESCRIPTION,DESCRIPTION,YES,YES,*Targeting designers with minimal coding exper...
...,...,...,...,...,...,...,...,...,...
135,sen_33,"The question-order effect (i.e., one question ...",LIMITATION,LIMITATION,OVERALL,LIMITATION,NO,NO,*The sequence of questions can influence respo...
136,sen_15,The main limitation is that the experiments ar...,LIMITATION,LIMITATION,RESEARCH GAP,LIMITATION,NO,NO,*The primary constraint lies in the fact that ...
137,sen_55,"However, special care must be taken to avoid b...",LIMITATION,LIMITATION,OTHER,LIMITATION,NO,NO,"*Nonetheless, precautions are necessary to pre..."
138,sen_10,"Hence, it is possible that a more detailed cal...",LIMITATION,LIMITATION,LIMITATION,LIMITATION,YES,YES,"*Consequently, a more comprehensive calibratio..."


In [ ]:
 df_copy = df.copy()
 df_copy


,Id,Sentence,Fco,Pablo,Paula,Vote,Unique,Unique_Vote,Synthetic_Sentence
0,RecSys 02 DES 4,in our recent work on music genre exploration ...,DESCRIPTION,DESCRIPTION,DESCRIPTION,DESCRIPTION,YES,YES,*In our latest research on musical genre explo...
1,RecSys 04 DES 3,in order to reflect the users complete spectru...,DESCRIPTION,DESCRIPTION,DESCRIPTION,DESCRIPTION,YES,YES,*To capture the diverse reading preferences of...
2,RecSys 05 DES 1,mcauley et al. [21] mine key attributes from t...,DESCRIPTION,DESCRIPTION,DESCRIPTION,DESCRIPTION,YES,YES,*[21] employs topic modeling techniques to ext...
3,CHI 01 DES 3,"beyond inspection, chickenfoot [7] allows end-...",DESCRIPTION,DESCRIPTION,DESCRIPTION,DESCRIPTION,YES,YES,"*Transcending mere examination, Chickenfoot [7..."
4,CHI 01 DES 5,aimed at designers who have limited coding kno...,DESCRIPTION,DESCRIPTION,DESCRIPTION,DESCRIPTION,YES,YES,*Targeting designers with minimal coding exper...
...,...,...,...,...,...,...,...,...,...
135,sen_33,"The question-order effect (i.e., one question ...",LIMITATION,LIMITATION,OVERALL,LIMITATION,NO,NO,*The sequence of questions can influence respo...
136,sen_15,The main limitation is that the experiments ar...,LIMITATION,LIMITATION,RESEARCH GAP,LIMITATION,NO,NO,*The primary constraint lies in the fact that ...
137,sen_55,"However, special care must be taken to avoid b...",LIMITATION,LIMITATION,OTHER,LIMITATION,NO,NO,"*Nonetheless, precautions are necessary to pre..."
138,sen_10,"Hence, it is possible that a more detailed cal...",LIMITATION,LIMITATION,LIMITATION,LIMITATION,YES,YES,"*Consequently, a more comprehensive calibratio..."


In [ ]:
df= df_copy.copy()
df

,Id,Sentence,Fco,Pablo,Paula,Vote,Unique,Unique_Vote,Synthetic_Sentence
0,RecSys 02 DES 4,in our recent work on music genre exploration ...,DESCRIPTION,DESCRIPTION,DESCRIPTION,DESCRIPTION,YES,YES,*In our latest research on musical genre explo...
1,RecSys 04 DES 3,in order to reflect the users complete spectru...,DESCRIPTION,DESCRIPTION,DESCRIPTION,DESCRIPTION,YES,YES,*To capture the diverse reading preferences of...
2,RecSys 05 DES 1,mcauley et al. [21] mine key attributes from t...,DESCRIPTION,DESCRIPTION,DESCRIPTION,DESCRIPTION,YES,YES,*[21] employs topic modeling techniques to ext...
3,CHI 01 DES 3,"beyond inspection, chickenfoot [7] allows end-...",DESCRIPTION,DESCRIPTION,DESCRIPTION,DESCRIPTION,YES,YES,"*Transcending mere examination, Chickenfoot [7..."
4,CHI 01 DES 5,aimed at designers who have limited coding kno...,DESCRIPTION,DESCRIPTION,DESCRIPTION,DESCRIPTION,YES,YES,*Targeting designers with minimal coding exper...
...,...,...,...,...,...,...,...,...,...
135,sen_33,"The question-order effect (i.e., one question ...",LIMITATION,LIMITATION,OVERALL,LIMITATION,NO,NO,*The sequence of questions can influence respo...
136,sen_15,The main limitation is that the experiments ar...,LIMITATION,LIMITATION,RESEARCH GAP,LIMITATION,NO,NO,*The primary constraint lies in the fact that ...
137,sen_55,"However, special care must be taken to avoid b...",LIMITATION,LIMITATION,OTHER,LIMITATION,NO,NO,"*Nonetheless, precautions are necessary to pre..."
138,sen_10,"Hence, it is possible that a more detailed cal...",LIMITATION,LIMITATION,LIMITATION,LIMITATION,YES,YES,"*Consequently, a more comprehensive calibratio..."


In [ ]:
import re
def extract_between_asterisks(text):
  # Use regex to find all text between asterisks
  return re.findall(r'\*(.*?)\*', text)

# Apply the extraction function and expand the results into new columns
df_expanded = df_copy['Synthetic_Sentence'].apply(lambda x: pd.Series(extract_between_asterisks(x)))

# Rename the columns as needed
df_expanded.columns = [f'Synthetic_Sentence_{i}' for i in range(1, len(df_expanded.columns) + 1)]

# Concatenate the original DataFrame with the new columns
result_df = pd.concat([df_copy, df_expanded], axis=1)

result_df

,Id,Sentence,Fco,Pablo,Paula,Vote,Unique,Unique_Vote,Synthetic_Sentence,Synthetic_Sentence_1,Synthetic_Sentence_2,Synthetic_Sentence_3,Synthetic_Sentence_4
0,RecSys 02 DES 4,in our recent work on music genre exploration ...,DESCRIPTION,DESCRIPTION,DESCRIPTION,DESCRIPTION,YES,YES,*In our latest research on musical genre explo...,In our latest research on musical genre explor...,Our recent study on music genre exploration [1...,In our latest work related to exploring music ...,Our recent research project on music genre exp...
1,RecSys 04 DES 3,in order to reflect the users complete spectru...,DESCRIPTION,DESCRIPTION,DESCRIPTION,DESCRIPTION,YES,YES,*To capture the diverse reading preferences of...,To capture the diverse reading preferences of ...,Recognizing that readers have varied interests...,With the goal of mirroring the full breadth of...,To reflect the diverse topical interests of re...
2,RecSys 05 DES 1,mcauley et al. [21] mine key attributes from t...,DESCRIPTION,DESCRIPTION,DESCRIPTION,DESCRIPTION,YES,YES,*[21] employs topic modeling techniques to ext...,[21] employs topic modeling techniques to extr...,The study by mcauley et al. [21] utilizes topi...,mcauley et al. [21] leverage topic extraction ...,The researchers in [21] apply topic extraction...
3,CHI 01 DES 3,"beyond inspection, chickenfoot [7] allows end-...",DESCRIPTION,DESCRIPTION,DESCRIPTION,DESCRIPTION,YES,YES,"*Transcending mere examination, Chickenfoot [7...","Transcending mere examination, Chickenfoot [7]...","Surpassing the confines of mere inspection, Ch...","Extending beyond the realm of mere inspection,...",Transcending the boundaries of mere inspection...
4,CHI 01 DES 5,aimed at designers who have limited coding kno...,DESCRIPTION,DESCRIPTION,DESCRIPTION,DESCRIPTION,YES,YES,*Targeting designers with minimal coding exper...,Targeting designers with minimal coding expert...,Designed for designers with restricted coding ...,Catering to designers with limited coding skil...,Aimed at designers with constrained coding abi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,sen_33,"The question-order effect (i.e., one question ...",LIMITATION,LIMITATION,OVERALL,LIMITATION,NO,NO,*The sequence of questions can influence respo...,The sequence of questions can influence respon...,One query may establish a context that guides ...,The order in which questions are presented can...,The question-order effect refers to the potent...
136,sen_15,The main limitation is that the experiments ar...,LIMITATION,LIMITATION,RESEARCH GAP,LIMITATION,NO,NO,*The primary constraint lies in the fact that ...,The primary constraint lies in the fact that t...,The study's scope is restricted to the stereot...,The experiments' foundation rests exclusively ...,The research's confines stem from its reliance...
137,sen_55,"However, special care must be taken to avoid b...",LIMITATION,LIMITATION,OTHER,LIMITATION,NO,NO,"*Nonetheless, precautions are necessary to pre...","Nonetheless, precautions are necessary to prev...",Particular attention should be paid to mitigat...,Careful measures must be implemented to circum...,Specific steps are required to eliminate the p...
138,sen_10,"Hence, it is possible that a more detailed cal...",LIMITATION,LIMITATION,LIMITATION,LIMITATION,YES,YES,"*Consequently, a more comprehensive calibratio...","Consequently, a more comprehensive calibration...",It is conceivable that a more thorough calibra...,A more intricate calibration procedure may pot...,"Therefore, the possibility exists that a more ..."


In [ ]:
df.to_csv('/content/sample_data/Sentence_Classification_Raters_SyntheticData_Original_v01.csv', index=False)
result_df.to_csv('/content/sample_data/Sentence_Classification_Raters_SyntheticData_Cleaned_v01.csv', index=False)


